In [3]:
pip install dash plotly pandas scikit-learn

In [4]:
# Run first: pip install dash plotly pandas scikit-learn

import dash
from dash import dcc, html, Input, Output
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

# =======================
# Load synthetic SBW data
# =======================
df = pd.read_csv("synthetic_sensor_data(1).csv")

# -----------------------
# Feature Engineering
# -----------------------
df['torque_error'] = df['torque_demand'] - df['torque_actual']
df['torque_rate'] = df['torque_actual'].diff() / df['time_s'].diff()
df['pressure_rate'] = df['clutch_pressure'].diff() / df['time_s'].diff()
df = df.fillna(0)

df['torque_error_rm'] = df['torque_error'].rolling(5).mean().fillna(0)
df['torque_rate_rm'] = df['torque_rate'].rolling(5).mean().fillna(0)

features = ['torque_error_rm', 'torque_rate_rm', 'pressure_rate', 'actuator_pos']
X = df[features].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# -----------------------
# Isolation Forest (Fault Detection)
# -----------------------
model = IsolationForest(contamination=0.05, random_state=42)
df['fault_flag'] = model.fit_predict(X_scaled)
faults = df[df['fault_flag'] == -1]
print(f"Detected {len(faults)} fault points")

# Downsample for performance
df_plot = df.iloc[::10, :]
faults_plot = faults.iloc[::10, :]

# =======================
# Dash App
# =======================
app = dash.Dash(__name__)
app.title = "Shift-by-Wire Dashboard"

# -----------------------
# App Layout
# -----------------------
app.layout = html.Div([
    html.H1("⚙️ Shift-by-Wire System Dashboard", style={'textAlign': 'center'}),

    html.Div([
        html.Label("Select Signal to Plot:"),
        dcc.Dropdown(
            id='signal-dropdown',
            options=[
                {'label': 'Torque Demand', 'value': 'torque_demand'},
                {'label': 'Torque Actual', 'value': 'torque_actual'},
                {'label': 'Actuator Position', 'value': 'actuator_pos'},
                {'label': 'Clutch Pressure', 'value': 'clutch_pressure'},
                {'label': 'Torque Error (Rolling Mean)', 'value': 'torque_error_rm'}
            ],
            value='torque_actual',
            clearable=False,
            style={'width': '50%'}
        )
    ], style={'padding': '10px'}),

    dcc.Graph(id='signal-plot'),

    html.Div(id='summary', style={'padding': '10px', 'fontSize': '16px', 'textAlign': 'center'})
])


# -----------------------
# Callback (Interactivity)
# -----------------------
@app.callback(
    [Output('signal-plot', 'figure'),
     Output('summary', 'children')],
    [Input('signal-dropdown', 'value')]
)
def update_plot(selected_signal):
    # Create figure
    fig = go.Figure()

    # Plot main signal
    fig.add_trace(go.Scatter(
        x=df_plot['time_s'],
        y=df_plot[selected_signal],
        mode='lines',
        name=f"{selected_signal}",
        line=dict(color='royalblue', width=1.2)
    ))

    # Overlay detected fault points
    if selected_signal in faults_plot.columns:
        fig.add_trace(go.Scatter(
            x=faults_plot['time_s'],
            y=faults_plot[selected_signal],
            mode='markers',
            name='Detected Fault',
            marker=dict(color='red', size=6, opacity=0.8, symbol='circle')
        ))

    # Layout customization
    fig.update_layout(
        title=f"📊 {selected_signal.replace('_', ' ').title()} vs Time (with Fault Overlay)",
        xaxis_title="Time (s)",
        yaxis_title=selected_signal.replace('_', ' ').title(),
        hovermode="x unified",
        template="plotly_white",
        width=1100,
        height=500
    )

    # Add interactive range slider
    fig.update_xaxes(rangeslider_visible=True)

    # Summary text
    mean_val = df[selected_signal].mean()
    std_val = df[selected_signal].std()
    total_faults = len(faults)
    summary = f"Mean: {mean_val:.2f} | Std: {std_val:.2f} | Total Faults Detected: {total_faults}"

    return fig, summary


# -----------------------
# Run Server
# -----------------------
if __name__ == '__main__':
    app.run(debug=True)


Detected 50000 fault points


<IPython.core.display.Javascript object>